In [19]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
# URL = 'https://www.q-net.or.kr/crf005.do?id=crf00505&jmCd=1320'
# tables = pd.read_html(URL)
# print(len(tables),'개의 테이블이 있습니다')
# tables[0]


# df=tables[0]

# df.to_csv("info.csv", index=False, encoding='utf-8-sig')


# file_name='abc.xlsx'
# f = open(file_name,'w', encoding='cp949', newline='')
# csv_writer = csv.writer(f)


# print(f'{file_name} 파일 저장 완료')


import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parse
from selenium.webdriver.chrome.service import Service

### KLPGA 기술 카테고리 크롤링

In [32]:
service = Service(executable_path='chromedriver.exe') 
options=webdriver.ChromeOptions()
driver=webdriver.Chrome(service=service,options=options)
driver.get('https://klpga.co.kr/web/record/publicRecordDetail')
time.sleep(1)

dfs = []
common_columns = None

for i in range(2008, 2024):
    driver.execute_script(f"document.getElementById('searchSeason').value={i};")
    driver.find_element(By.CSS_SELECTOR, 'a.nav-link[data-menu1="Score"]').click()

    time.sleep(2)
    html = driver.page_source
    bs = BeautifulSoup(html, "html.parser")
    table = bs.select_one("table.table.table-record.table-hover")

    time.sleep(2)

    table_data = parse.make2d(table)                          # 2d로 정렬
    df = pd.DataFrame(table_data)

    if common_columns is None:
        common_columns = df.iloc[0].tolist()                    # 불러오는 테이블들이 공통 컬럼을 사용할 수 있게
        df.columns = common_columns                             # 첫번째 행을 컬럼으로 사용

    # reset_index 후 테이블들의 첫번째 행 삭제
    df = df[1:].reset_index(drop=True)

    time.sleep(2)

    dfs.append(df)

# 하나의 데이터프레임으로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 파일 저장
merged_df.to_csv('golfscore.csv', encoding='utf-8-sig')


### KLPGA 스코어 카테고리 크롤링

In [33]:
service = Service(executable_path='chromedriver.exe') 
options=webdriver.ChromeOptions()
driver=webdriver.Chrome(service=service,options=options)
driver.get('https://klpga.co.kr/web/record/publicRecordDetail')
time.sleep(1)

dfs = []
common_columns = None

for i in range(2008, 2024):
    driver.execute_script(f"document.getElementById('searchSeason').value={i};")
    driver.find_element(By.CSS_SELECTOR, 'a.nav-link[data-menu1="Tech"]').click()

    time.sleep(2)
    html = driver.page_source
    bs = BeautifulSoup(html, "html.parser")
    table = bs.select_one("table.table.table-record.table-hover")

    time.sleep(2)

    table_data = parse.make2d(table)  # 2-dimensional array
    df = pd.DataFrame(table_data)

    if common_columns is None:
        common_columns = df.iloc[0].tolist()  # Use the first row as common columns
        df.columns = common_columns  # Set the columns using the first row

    # Skip the first row (used for columns) and reset the index
    df = df[1:].reset_index(drop=True)

    time.sleep(2)

    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Print or use the merged DataFrame as needed
merged_df.to_csv('golfTech.csv', encoding='utf-8-sig')


In [35]:
file1="golfscore.csv"
file2="golfTech.csv"

scoreDF=pd.read_csv(file1)
techDF=pd.read_csv(file2)

In [38]:
scoreDF = scoreDF.groupby('선수명').mean().reset_index()
techDF = techDF.groupby('선수명').mean().reset_index()

In [39]:
merged_df = pd.merge(scoreDF, techDF, on='선수명', how='outer')
merged_df

,선수명,Unnamed: 0_x,FAV_x,순위_x,평균타수,파3성적,파4성적,파5성적,평균버디,버디율,...,파4평균퍼팅,파5평균퍼팅,그린적중률,파3그린적중률,파4그린적중률,파5그린적중률,벙커세이브율,리커버리율,드라이빙 지수,아이언샷 지수
0,Ainil BAKAR,2127.500000,NaN,NaN,75.882525,3.117075,4.262450,5.189400,2.3125,12.845275,...,1.792975,1.863575,60.879350,52.082500,57.454375,73.124650,NaN,39.942925,NaN,NaN
1,Ajira NUALRAKSA,2315.666667,NaN,NaN,78.095200,3.375000,4.375167,5.163333,1.8700,10.384433,...,1.675467,1.577767,51.320733,45.417500,42.649775,52.432500,NaN,30.204200,NaN,NaN
2,Ajira Nualraksa,1093.800000,NaN,NaN,77.350000,3.220000,4.344000,5.262000,1.3300,7.410000,...,1.776667,1.666667,66.670000,56.526667,60.556667,65.833333,NaN,33.330000,NaN,NaN
3,Albane Valenzuela,3273.000000,NaN,NaN,72.250000,2.875000,4.150000,4.812500,2.5000,13.888900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.111100,NaN,NaN
4,Alena Sharp,2753.000000,NaN,NaN,70.750000,2.937500,4.000000,4.750000,3.0000,16.666700,...,1.700000,1.533300,73.611100,75.000000,75.000000,68.750000,NaN,73.684200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,김영,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1395,송아리,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1396,신이솔,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1397,유선영,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
merged_df.to_csv('mergegolf.csv', encoding='utf-8-sig')

In [42]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   선수명           1399 non-null   object 
 1   Unnamed: 0_x  1374 non-null   float64
 2   FAV_x         0 non-null      float64
 3   순위_x          525 non-null    float64
 4   평균타수          1373 non-null   float64
 5   파3성적          1374 non-null   float64
 6   파4성적          1374 non-null   float64
 7   파5성적          1374 non-null   float64
 8   평균버디          1107 non-null   float64
 9   버디율           1107 non-null   float64
 10  파3평균버디        1148 non-null   float64
 11  파4평균버디        1286 non-null   float64
 12  파5평균버디        1238 non-null   float64
 13  홀인원           169 non-null    float64
 14  이글            550 non-null    float64
 15  파브레이크율        1104 non-null   float64
 16  Unnamed: 0_y  1307 non-null   float64
 17  FAV_y         0 non-null      float64
 18  순위_y          525 non-null  